In [211]:
import numpy as np
import pandas as pd
import warnings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

warnings.filterwarnings('ignore')

def take_a_look (data_to_look):
    print(data_to_look.info())
    print()
    print(data_to_look.describe())

def correlation_columns (data, col_1, col_2):
    corr_value = data[col_1].corr(data[col_2])
    return round(corr_value, 4) * 100

def get_subset_threshold(data, column, threshold, condition):
    """
    Parameters:
    data: data to filter -> Dtype: data frame
    column: filter based on this column -> Dtyle: string
    threshold: value above/below you want to filter the data -> Dtype: float
    condition: takes two possible values (greater than or equal to / less than) -> Dtype: string
    """
    if condition == '>=':
        mask = data[column] >= threshold
        data_mask = data[mask]

    else:
        mask = data[column] < threshold
        data_mask = data[mask]

    return data_mask

def get_subset_exact(data, column, column_value):
    """
    This function generates subsets based on categorical column value.
    Parameters:
    data: data to filter -> Dtype: data frame
    column: filter based on this column -> Dtype: string
    column value: values that are equal to this are filtered -> Dtype: string
    """

    mask = data[column] == column_value
    data_mask = data[mask]

    return data_mask
    

In [213]:
data = pd.read_csv('Employees_clean.csv')
data.head()

,Gender,Age,Department,Country,Center,Annual Salary,Job Rate,Sick Leaves,Unpaid Leaves,Overtime Hours
0,Female,58,Environmental Compliance,Egypt,Main,12538.8,5.0,0,0,9
1,Male,44,Manufacturing,Egypt,West,24305.4,5.0,3,0,43
2,Female,56,Account Management,Egypt,Main,36849.6,5.0,2,0,3
3,Male,59,Quality Control,Egypt,Main,40458.6,5.0,6,1,9
4,Female,58,Facilities/Engineering,Egypt,Main,15184.8,4.5,4,0,46


In [5]:
male_employees = get_subset_exact(data, "Gender", "Male")
female_employees  = get_subset_exact(data, "Gender", "Female")

gender_counts = data['Gender'].value_counts()
gender_ratio = gender_counts[0]/gender_counts[1]

print(f"The ratio of {gender_counts[0]} to {gender_counts[1]} employees is {gender_ratio}")

The ratio of 449 to 240 employees is 1.8708333333333333


In [7]:
data_manfufacturing = get_subset_exact(data, 'Department', 'Manufacturing')
print(data_manfufacturing['Gender'].value_counts())

Gender
Male      95
Female    45
Name: count, dtype: int64


In [9]:
unique_departments = data['Department'].unique()
male_employees_count = []
female_employees_count = []

for i in unique_departments:
    data_subset = get_subset_exact(data, 'Department', i)
    
    data_subset_gender_count = data_subset['Gender'].value_counts()

    male_employees_count.append(data_subset_gender_count[0])

    if data_subset_gender_count[0] != len(data_subset):
        female_employees_count.append(data_subset_gender_count[1])
    else:
        female_employees_count.append(0)


In [11]:
male_employees_count = np.array(male_employees_count)
female_employees_count = np.array(female_employees_count)

In [13]:
idx_highest_male_count = np.argmax(male_employees_count)
print("The department with the highest male employees is:", unique_departments[idx_highest_male_count])

idx_highest_female_count = np.argmax(female_employees_count)
print("The department with the highest female employees is:", unique_departments[idx_highest_female_count])

The department with the highest male employees is: Manufacturing
The department with the highest female employees is: Manufacturing


In [15]:
idx_lowest_male_count = np.argmin(male_employees_count)
print("The department with the lowest male employees is:", unique_departments[idx_lowest_male_count])

idx_lowest_female_count = np.argmin(female_employees_count)
print("The department with the lowest female employees is:", unique_departments[idx_lowest_male_count])

The department with the lowest male employees is: Research/Development
The department with the lowest female employees is: Research/Development


In [37]:
unique_departments = data['Department'].unique()
data_manufacturing = get_subset_exact(data, 'Department', 'Manufacturing')

avg_salary_manufacturing = np.around(np.mean(data_manufacturing['Annual Salary']), 2)

print("The average salary of Manufacturing is:", avg_salary_manufacturing)

The average salary of Manufacturing is: 24172.1


In [109]:
salary_data = []

for i in unique_departments:
    data_subset = get_subset_exact(data, 'Department', i)

    data_subset_mean = np.around(np.mean(data_subset['Annual Salary']), 2)

    salary_data.append(data_subset_mean)


In [111]:
salary_dict = {'Department': unique_departments, 'Average Salary': salary_data}

table_salary_department = pd.DataFrame(salary_dict)

table_salary_department.sort_values(by = 'Average Salary', ignore_index= True, ascending= False, inplace= True)

In [113]:
# get average country salaries
country_salary_data = []
unique_countries = data['Country'].unique()

for i in unique_countries:
    data_subset = get_subset_exact(data, 'Country', i)

    data_subset_mean = np.around(np.mean(data_subset['Annual Salary']), 2)

    country_salary_data.append(data_subset_mean)

In [191]:
# make and sort dataframe to get high and lowest country avg salaries
country_salary_dict = {'Country': unique_countries, 'Average Salary': country_salary_data}

table_country_salary = pd.DataFrame(country_salary_dict)
table_country_salary.sort_values(by = 'Average Salary', ignore_index= True, ascending= False, inplace= True)

print(f"Country with the highest salary is {table_country_salary['Country'][0]} with a salary of {table_country_salary['Average Salary'][0]}")

print(f"Country with the lowest salary is {table_country_salary['Country'][len(table_country_salary) - 1]}"
f" with a salary of {table_country_salary['Average Salary'][len(table_country_salary) - 1]}")

Country with the highest salary is Egypt with a salary of 24955.48
Country with the lowest salary is Lebanon with a salary of 23245.69


In [215]:
def salary_report(data, feature):
    """
    This function generates a report for average salaries per unique values in a given feature.
    Parameters:
    data: data to generate the report -> Dtype: data frame
    feature: column name on which the report will be generated -> Dtyle: string
    """
    
    salaries = []

    unique_features = data[feature].unique()

    for i in unique_features:
        data_feature = get_subset_exact(data, feature, i)

        salary = np.around(np.mean(data_feature['Annual Salary']), 2)
        salaries.append(salary)
    salary_report = pd.DataFrame({feature: unique_features, 'Annual Salary': salaries})
    
    return salary_report

In [217]:
salary_report(data, 'Country')

,Country,Annual Salary
0,Egypt,24955.48
1,United Arab Emirates,24870.95
2,Saudi Arabia,24173.74
3,Syria,24453.41
4,Lebanon,23245.69
